In [1]:
from sympy import mod_inverse

In [2]:
def double_point(x,y,a,p):
    m = (3 * x**2 + a) * mod_inverse(2 * y, p) % p
    x3 = (m**2 - x - x) % p
    y3 = (m * (x - x3) - y) % p
    return (x3,y3)

In [12]:
def add_point(x1,x2,y1,y2,a,p):
    m = (y2 - y1) * mod_inverse(x2 - x1, p) % p
    x3 = (m**2 - x1 - x2) % p
    y3 = (m * (x1 - x3) - y1) % p
    return (x3,y3)

In [13]:
x=18446744073709551616
y=663271545314189690754141204937684797586435551979845320856062
a=2024
p=987308279214583009270517326540245456902721241277308160881297
n=0
# print(add_point(3,6,8,5,1,11))
print(double_point(x,y,a,p))

(431533405883826640720265106051064233758762656048974554568048, 54633359396906083705826900289904079201648170645043617259856)


In [14]:
x1, y1 = 18446744073709551616, 663271545314189690754141204937684797586435551979845320856062
x2, y2= 431533405883826640720265106051064233758762656048974554568048,54633359396906083705826900289904079201648170645043617259856
a = 2024
modulus = 987308279214583009270517326540245456902721241277308160881297
print(add_point(x1,x2,y1,y2,a,modulus))

(110380820198006169710562005071197260234008568439753724561357, 598952754833104422504771358959288688709570873572530135426598)


In [16]:
def point_multiplication(P1, P2,a,modulus):
    if P1 is None:
        return P2
    if P2 is None:
        return P1
        
    x1, y1 = P1
    x2, y2 = P2

    if (x1, y1) == (x2, -y2 % modulus):
        return None  # Point at infinity
        
    if P1 != P2:
        return add_point(x1,x2,y1,y2,a,modulus)
    else:
        return double_point(x1,y1,a,modulus)
    

def find_order(x,y,a,modulus):
    P = (x, y)
    order = 1
    Q = P

    while Q is not None:
        order += 1
        Q = point_multiplication(Q, P, a, modulus)
    
    print(C, order)

#Values 
x, y = 4868, 514546
a = 169942
modulus = 883433

find_order(x,y,a,modulus)

787086 23268


In [17]:
x, y = 4868, 514546
a = 169942
modulus = 883433

C = (y**2 - x**3 - a*x) % modulus

def elliptic_add(P1, P2, a, modulus):
    """Add two points P1 and P2 on an elliptic curve over a finite field."""
    if P1 is None:
        return P2
    if P2 is None:
        return P1

    x1, y1 = P1
    x2, y2 = P2

    if (x1, y1) == (x2, -y2 % modulus):
        return None  # Point at infinity

    if P1 != P2:
        # Slope of the line between P1 and P2
        m = (y2 - y1) * mod_inverse(x2 - x1, modulus) % modulus
    else:
        # Slope of the tangent line at P1
        m = (3 * x1**2 + a) * mod_inverse(2 * y1, modulus) % modulus

    x3 = (m**2 - x1 - x2) % modulus
    y3 = (m * (x1 - x3) - y1) % modulus

    return (x3, y3)

# Find the order of P
P = (x, y)
order = 1
Q = P

while Q is not None:
    order += 1
    Q = elliptic_add(Q, P, a, modulus)

print(C, order)


787086 23268


In [39]:
x, y = 4868, 514546
a = 169942
p = 883433

def point_doubling_algo(x,y,a,p,m):
    li=[]
    i=1
    li.append([(x,y),i])
    xn,yn=x,y
    i=2
    while i<m:
        xn,yn=double_point(xn,yn,a,p)
        li.append([(xn,yn),i])
        i*=2
        
    print(li)
    index=len(li)-1
    sum_list=[]
    while m!=0:
        if li[index][1]<=m:
            m=m-li[index][1]
            sum_list.append(index)
        else:
            index-=1
    for j in sum_list:
        print(li[j][1])
    j=sum_list[0]
    xn,yn=li[j][0] 
    
    for i in range(1,len(sum_list)):
        xadd,yadd=li[sum_list[i]][0]
        xn,yn=add_point(xn,xadd,yn,yadd,a,p)
        print(xn,yn)
    return (xn,yn)
point_doubling_algo(x,y,a,p,207)


[[(4868, 514546), 1], [(642327, 611915), 2], [(436852, 410338), 4], [(342762, 649309), 8], [(762840, 22384), 16], [(59863, 702460), 32], [(812720, 565849), 64], [(643615, 245173), 128]]
128
64
8
4
2
1
548922 768898
689921 765112
410672 137016
797136 36212
701748 111798


(701748, 111798)

In [36]:
# Decompressing a point
from sympy import nthroot_mod

# Constants for the elliptic curve
a = 7999764
b = 7853170
p = 38902957  # The modulus

# The x-coordinate and flag from the compressed point
x = 5842104
flag = 1  # Indicates that y is odd

# Compute the right-hand side of the equation
rhs = (x**3 + a*x + b) % p

# Find the modular square roots
y_roots = nthroot_mod(rhs, 2, p, True)

# Select the y-coordinate that is odd
y = [y for y in y_roots if y % 2 == flag][0]  # Select the odd root

print(y)


4877397


In [1]:
from sympy import nthroot_mod

def decompress_point(x, flag, a, b, p):
    """
    Decompress an elliptic curve point.
    
    Parameters:
    x (int): x-coordinate of the compressed point
    flag (int): Indicates whether the y-coordinate should be odd (1) or even (0)
    a (int): Coefficient a of the elliptic curve equation
    b (int): Coefficient b of the elliptic curve equation
    p (int): Prime modulus of the finite field
    
    Returns:
    (int, int): The decompressed point (x, y)
    """
    # Compute the right-hand side of the elliptic curve equation
    rhs = (x**3 + a*x + b) % p
    
    # Find the modular square roots
    y_roots = nthroot_mod(rhs, 2, p, True)
    
    # Select the y-coordinate that matches the parity indicated by the flag
    y = [y for y in y_roots if y % 2 == flag][0]
    
    return x, y

def compress_point(x, y):
    """
    Compress an elliptic curve point.
    
    Parameters:
    x (int): x-coordinate of the point
    y (int): y-coordinate of the point
    
    Returns:
    (int, int): The compressed point (x, flag), where flag indicates whether y is odd (1) or even (0)
    """
    # Determine the parity of the y-coordinate
    flag = y % 2
    
    return x, flag

# Example usage:
a = 7999764
b = 7853170
p = 38902957

# Given compressed point data
x_compressed = 5842104
flag_compressed = 1  # Indicates that y is odd

# Decompress the point
x_decompressed, y_decompressed = decompress_point(x_compressed, flag_compressed, a, b, p)
print(f"Decompressed Point: ({x_decompressed}, {y_decompressed})")

# Compress the point
x_compressed, flag_compressed = compress_point(x_decompressed, y_decompressed)
print(f"Compressed Point: ({x_compressed}, {flag_compressed})")


Decompressed Point: (5842104, 4877397)
Compressed Point: (5842104, 1)
